In [6]:
import os, sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Project root added to path:", PROJECT_ROOT)


Project root added to path: C:\Users\krish\MIMIC\mimic_nlp_project


In [7]:
import sys
print(sys.executable)
import pandas as pd
print("pandas", pd.__version__)


C:\Users\krish\miniconda3\envs\mimic-nlp\python.exe
pandas 2.3.3


In [8]:
# 01_data_ingest.ipynb - Demo dataset creation

import os
import pandas as pd

# Project folder and data folder
PROJECT_ROOT = os.getcwd()
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
os.makedirs(DATA_DIR, exist_ok=True)

# Creating a small demo CSV
demo_path = os.path.join(DATA_DIR, 'demo_notes.csv')

if not os.path.exists(demo_path):
    demo = pd.DataFrame({
        'subject_id': ['1','2'],
        'hadm_id': ['10','20'],
        'category': ['Discharge summary','Discharge summary'],
        'text': [
            'Patient has history of diabetes and hypertension. Discharged on metformin.',
            'No prior smoking. Obese with BMI 32. Plans for follow-up.'
        ]
    })
    demo.to_csv(demo_path, index=False)
    print("Demo CSV created at:", demo_path)
else:
    print("Demo CSV already exists at:", demo_path)

# Load and preview
df_demo = pd.read_csv(demo_path)
df_demo.head()


Demo CSV already exists at: C:\Users\krish\MIMIC\mimic_nlp_project\notebooks\data\demo_notes.csv


,subject_id,hadm_id,category,text
0,1,10,Discharge summary,Patient has history of diabetes and hypertensi...
1,2,20,Discharge summary,No prior smoking. Obese with BMI 32. Plans for...


In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("sentencizer")


In [10]:
import re
import spacy

# Loading spaCy model
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("sentencizer")  # splits text into sentences

# Simple text cleaning function
def clean_text(text):
    text = re.sub(r"\s+", " ", text)  # remove extra spaces
    return text.strip()

# Sentence splitting function
def split_sentences(text):
    doc = nlp(text)
    return [s.text.strip() for s in doc.sents if len(s.text.strip()) > 10]

# Apply to demo dataset
df_demo['clean_text'] = df_demo['text'].apply(clean_text)
df_demo['sentences'] = df_demo['clean_text'].apply(split_sentences)
df_demo.head()


,subject_id,hadm_id,category,text,clean_text,sentences
0,1,10,Discharge summary,Patient has history of diabetes and hypertensi...,Patient has history of diabetes and hypertensi...,[Patient has history of diabetes and hypertens...
1,2,20,Discharge summary,No prior smoking. Obese with BMI 32. Plans for...,No prior smoking. Obese with BMI 32. Plans for...,"[No prior smoking., Obese with BMI 32., Plans ..."


In [11]:
from src.preprocessing import clean_text, split_sentences
